In [7]:
!python -m wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
!python -m wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_prep.txt


Saved under dbpedia_sample_abstract_20k_unprep.txt

Saved under dbpedia_sample_abstract_20k_prep.txt


In [1]:
#Importing what we need
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file, TopicModelDataPreparation
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamodel 
import os
import numpy as np
import pickle

In [6]:
with open("dbpedia_sample_abstract_20k_prep.txt","r") as f1:
    text_training_processed = [line.strip() for line in f1.readlines()]
text_training_processed[0]
with open("dbpedia_sample_abstract_20k_unprep.txt","r") as f2:
    text_training_not_processed = [line.strip() for line in f2.readlines()]
assert len(text_training_processed) == len(text_training_not_processed)

'mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry'

In [7]:
#Split the documents in training and testing
index = 15000
training_documents = text_training_processed[0:index]
un_training_documents = text_training_not_processed[0:index]
testing_documents = text_training_processed[index:]
un_testing_documents = text_training_not_processed[index:]

In [ ]:
#Creating the Training Dataset
tp = TopicModelDataPreparation("bert-base-nli-mean-tokens")
training_dataset = tp.create_training_set(un_training_documents,training_documents)

In [ ]:
#Training our Combined Contextualized Topic Model
ctm = CombinedTM(input_size=len(tp.vocab), bert_input_size=768, num_epochs=100, n_components=50)
ctm.fit(training_dataset) 

In [ ]:
#Saving the model
ctm.save(models_dir="./")

In [ ]:
#Loading the Model
del ctm

In [ ]:
ctm = CombinedTM(input_size=len(tp.vocab), bert_input_size=768, num_epochs=100, n_components=50)
ctm.load("contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99/",
                                                                                                      epoch=26)

In [ ]:
#Look at our topics
get_topic_lists
ctm.get_topic_lists(5)

In [ ]:
#Using the testset
testing_dataset = tp.create_test_set(un_testing_documents,testing_documents)
predictions = ctm.get_doc_topic_distribution(testing_dataset, n_samples=10)
topic_index = np.argmax(predictions[15])
ctm.get_topic_lists(5)[topic_index]